# Move geometry from existing layer to new layer

## Imports

In [1]:
import os
from ansys.edb.database import Database
from ansys.edb.layer import StackupLayer, LayerType
from ansys.edb.layout import Cell, CellType
from ansys.edb.net import Net
from ansys.edb.primitive import Circle, Text, Rectangle, RectangleRepresentationType
from ansys.edb.session import session, launch_session

## Configs

In [2]:
# RPC_SERVER = None if separate server is already running, Root dir of server executable otherwise.
RPC_SERVER = None
ROOT = os.getcwd()
EDB_FILE = os.path.join(ROOT, "move_geometry_from_existing_layer_to_new_layer.aedb")

## Launch a session

In [3]:
# Make sure old one has been disconnected prior to starting a new one, if any
session = launch_session(RPC_SERVER, 50051)

## Basic setup of empty EDB

In [4]:
"""Create Database, Cell, Layout, Net"""
db = Database.create(EDB_FILE)
cell = Cell.create(db, CellType.CIRCUIT_CELL, "EMDesign1")
layout = cell.layout
net = Net.create(layout, 'GND')

id: 1



## Setup Layers for cell

In [5]:
"""Create layers, add them to layer collection and retrieve them again."""

# Layers in layer collection
layer_gnd = StackupLayer.create('Ground', layer_type=LayerType.SIGNAL_LAYER, thickness=0.0001, elevation=0, material="copper")
layer_1 = StackupLayer.create('NewLayer', LayerType.SIGNAL_LAYER, thickness=0.0001, elevation=0.0001, material="copper")

# Layer Collection Setup
lc = layout.layer_collection
lc.add_layers([layer_gnd, layer_1])

# Retrieve layer collection and layers
layer_gnd = lc.find_by_name('Ground')
layer_1 = lc.find_by_name('NewLayer')

# Priint layers created
print([(l.name) for l in lc.get_layers() ])

['NewLayer', 'Ground']


## Create geometry

In [6]:
"""Create Primitive objects in different layers"""

# Multiply each value with a resolution
um = lambda val : val * 1e-6

text = Text.create(layout=layout,layer=layer_gnd, center_x=um(0.0), center_y=um(0.0), text="my Text")

rect = Rectangle.create(layout=layout,
                        layer=layer_gnd,
                        net=net,
                        rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                        param1=um(10),
                        param2=um(10),
                        param3=um(20),
                        param4=um(20),
                        corner_rad=0.0,
                        rotation=0.0
                        )

rect_2 = Rectangle.create(layout=layout,
                          layer=layer_gnd,
                          net=net,
                          rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                          param1=um(0.0),
                          param2=um(0.0),
                          param3=um(30),
                          param4=um(30),
                          corner_rad=0.0,
                          rotation=0.0
                          )

circle = Circle.create(layout=layout,layer=layer_gnd,net=net, center_x=um(0.0), center_y=um(0.0), radius=um(34.0))

#Retrieve the primitives
primitives_created = layout.primitives

# Print primitives created and their layers
print([(p.primitive_type.name, p.layer.name) for p in primitives_created ])

[('TEXT', 'Ground'), ('RECTANGLE', 'Ground'), ('RECTANGLE', 'Ground'), ('CIRCLE', 'Ground')]


## Move Primitive objects to a new layer

In [7]:
"""Move Primitive objects to NewLayer"""

for primitive in primitives_created:
    primitive.layer = layer_1

# Print primitives and their layers
print([(p.primitive_type.name, p.layer.name) for p in primitives_created ])

[('TEXT', 'NewLayer'), ('RECTANGLE', 'NewLayer'), ('RECTANGLE', 'NewLayer'), ('CIRCLE', 'NewLayer')]


# Save database

In [8]:
"""Save database if you wish."""

db.save()

# Disconnect from database

In [9]:
"""Don't forget to disconnect it!"""

session.disconnect()